## Deeper Model

### Residual Block

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.nn import init
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch import nn, optim
import torchvision
from torchvision import transforms

%load_ext autoreload
%autoreload 2
import utils

In [34]:
class ResidualBlock(nn.Module):

    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_ch)

        self.shortcut = nn.Sequential()

        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )

    def forward(self, X):
        out = F.relu(self.bn1(self.conv1(X)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(X)
        out = F.relu(out)
        return out

class PreActBlock(nn.Module):

    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_ch)
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=False)
        self.shortcut = nn.Sequential()

        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False),
            )

    def forward(self, X):
        out = self.conv1(F.relu(self.bn1(X)))
        out = self.conv2(F.relu(self.bn2(out)))
        out += self.shortcut(X)
        return out

In [30]:
resblock = ResidualBlock(3, 64, stride=2)
preact = PreActBlock(3, 64, stride=1)

In [31]:
preact

PreActBlock(
  (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (shortcut): Sequential(
    (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
  )
)

In [32]:
X = torch.randn(1, 3, 32, 32)
output = preact(X)

In [33]:
output.shape

torch.Size([1, 64, 32, 32])

### Bottleneck構造

In [37]:
class Bottleneck(nn.Module):

    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.expansion_factor = 4
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(out_ch)
        
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_ch)

        self.conv3 = nn.Conv2d(out_ch, out_ch*self.expansion_factor, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_ch*self.expansion_factor)

        self.shortcut = nn.Sequential()

        if stride != 1 or in_ch != out_ch*self.expansion_factor:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch*self.expansion_factor, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_ch*self.expansion_factor)
            )

    def forward(self, X):
        out = F.relu(self.bn1(self.conv1(X)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(X)
        out = F.relu(out)
        return out

In [39]:
bottleneck = Bottleneck(256, 64, stride=1)
X = torch.randn(1, 256, 32, 32)
output = bottleneck(X)
output.shape

torch.Size([1, 256, 32, 32])

In [40]:
output2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)(X)

In [41]:
output2.shape

torch.Size([1, 256, 32, 32])

### Inception

In [56]:
class InceptionModule(nn.Module):

    def __init__(self, in_ch, out_ch1, out_ch3, out_ch5, out_ch_pool):
        super().__init__()

        # 1x1
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_ch, out_ch1, kernel_size=1),
            nn.BatchNorm2d(out_ch1),
            nn.ReLU(),
        )

        # 3x3
        self.branch2 = nn.Sequential(
            # point-wise
            nn.Conv2d(in_ch, out_ch3, kernel_size=1),
            nn.BatchNorm2d(out_ch3),
            nn.ReLU(),
            # 3x3 conv
            nn.Conv2d(out_ch3, out_ch3, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch3),
            nn.ReLU(),
        )

        # 5x5
        self.branch3 = nn.Sequential(
            # point-wise
            nn.Conv2d(in_ch, out_ch5, kernel_size=1),
            nn.BatchNorm2d(out_ch5),
            nn.ReLU(),
            # 5x5 conv
            nn.Conv2d(out_ch5, out_ch5, kernel_size=5, padding=2),
            nn.BatchNorm2d(out_ch5),
            nn.ReLU(),
        )

        # pooling
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_ch, out_ch_pool, kernel_size=1),
            nn.BatchNorm2d(out_ch_pool),
            nn.ReLU(),
        )
        
    def forward(self, X):
        branch1 = self.branch1(X)
        branch2 = self.branch2(X)
        branch3 = self.branch3(X)
        branch4 = self.branch4(X)

        return torch.cat([branch1, branch2, branch3, branch4], dim=1)

In [57]:
X = torch.randn(16, 192, 28, 28)
module = InceptionModule(192, 64, 128, 32, 32)
out = module(X)
print(out.shape)

torch.Size([16, 256, 28, 28])


In [58]:
module

InceptionModule(
  (branch1): Sequential(
    (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (branch2): Sequential(
    (0): Conv2d(192, 128, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (branch3): Sequential(
    (0): Conv2d(192, 32, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (branch4): Sequential(
    (0): Ma

In [52]:
torch.cat([a, b], dim=1).shape

torch.Size([1, 6])